<a href="https://colab.research.google.com/github/lrwerneck/ManimEngineClaude/blob/main/ClaudeManimEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MANIM ENGINE POWERED BY CLAUDE OPUS

Run the cell below to install the necessary libraries and dependencies into our run time. The process will be slightly different if you are not doing this in Google Colab. Please reference https://docs.manim.community/en/stable/installation.html if that is the case.

You will need to restart the session after running the cell for the changes to take affect.

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install opencv-python
!pip install anthropic
# !pip install IPython --upgrade

To first check that manim was install correctly:

In [ ]:
from manim import *

Manim Community v0.18.0

We will extract the last frame of each section so that we can provide Opus with references to what the output is looking like.

In [ ]:
import cv2
import os

def extract_last_frame(mp4_file:str, output_folder = "section_frames"):
  """
  Function to extract the last frame from each section of our animation.
  Takes as input the relative file path for the section's mp4 file and outputs a png file
  to section_frames of the last frame of the animation.

  Example Usage : extract_last_frame(mp4_file = "/content/media/videos/content/720p30/sections/SquareToCircle_0001.mp4")

  """

  # Check that the output folder exists and if not we create it
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)

  video = cv2.VideoCapture(mp4_file)
  total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  video.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)

  success, last_frame = video.read()

  if success:
    # Extract the filename for the input file path
    filename = os.path.basename(mp4_file)

    # Replace the file extension from .mp4 to .png
    output_filename = filename.replace(".mp4", ".png")

    output_file = os.path.join(output_folder, output_filename)

    cv2.imwrite(output_file, last_frame)

    print("Last frame saved successfully.")

  else:
      print("Failed to read the last frame.")

  video.release()

Set your Anthropic API Key as an Environment Variable

In [ ]:
import os
os.environ["ANTHROPIC_API_KEY"] = "" # <- Enter your Anthropic API Key here

In [ ]:
import base64

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
        return base64_string

In [ ]:
from anthropic import Anthropic

client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
MODEL_NAME = "claude-3-opus-20240229"

Your task is to create a one-page website based on the given specifications, delivered as an HTML file with embedded JavaScript and CSS. The website should incorporate a variety of engaging and interactive design features, such as drop-down menus, dynamic text and content, clickable buttons, and more. Ensure that the design is visually appealing, responsive, and user-friendly. The HTML, CSS, and JavaScript code should be well-structured, efficiently organized, and properly commented for readability and maintainability.

In [ ]:
SYSTEM_MESSAGE = """
Your task is to create an animated visualization for the User's define topic, delievered through as Manim code. The animation should incorporate a variety of engaging animations and objects.
Ensure that the design is visually appealing, descriptive and most importantly accurate.
The Python manim code should be well-structured, efficiently organized, and properly commented for readability and maintainability.
Before every transition between important framesin the animation, self.next_section("ADD SHORT DESCRIPTION OF NEXT SCENE HERE") is required to be present throughout our Scene Class.
This process will be done in steps. The steps are as follows:
<Steps>
1. Provide a written description of the scene first within the tags <Written Descritption></Writte_ Descritption>.
2. Using the written description of the scene and user suggestions, write an abstract pseudocode that we will base the manim code on. The psuedocode should indicate specifcally where transitions between important frames are occuring.
3. Given the abstract pseudocode and our written description of the animation, write the manim scene class to produce our desired animation. Remember to use  self.next_section() where the pseudocode indicates.  Put this code in <rough_draft_animation></rough_draft_animation>.
4. Given the images of the main frames of the animation, confirm that everything is in within view of the camera and is visualized as intended. If the images appear satisfactory reply with the final manim code in <final_draft></final_draft>. If the images are not satisfactory state 'Revisions needed' and state the issues in <adjustments_needed></adjustments_needed>.
5. If the results are deemed unsatisfactory by the user, return to steps 2 or 3 depending on the issues. After the results are confirmed to be satisfactory by the user return the final animation code in <animated_scene></animated_scene>.
</Steps>
"""

In [ ]:
initial_prompt = "Create an animation of a simple Feed forward Neural Network with an input layer, hidden layer and output layer. Start by doing step 1."

In [ ]:
step_1 = client.messages.create(
    model= MODEL_NAME,
    system=SYSTEM_MESSAGE,
    messages=[
        {"role": "user", "content": initial_prompt}
    ],
    max_tokens=4096
)

In [ ]:
print(step_1.content[0].text)

Here is the written description of the scene:
<Written Descritption>
The animation will depict a simple feed-forward neural network consisting of an input layer, a hidden layer, and an output layer. The scene will start by showing the input layer with multiple nodes. Lines will then animate from the input nodes to the nodes in the hidden layer, representing the flow of data. The hidden layer nodes will light up, indicating the processing of data. Finally, lines will animate from the hidden layer nodes to the output layer node(s). The output layer will light up and the final output will be displayed. Labels will clearly identify each layer of the network.
</Written Descritption>


In [ ]:
step_2 = client.messages.create(
    model= MODEL_NAME,
    system=SYSTEM_MESSAGE,
    messages=[
        {"role": "user", "content": initial_prompt},
        {"role" :"assistant", "content": step_1.content[0].text},
        {"role":"user","content":"The written_description looks good. Please continue with step 2."}
    ],
    max_tokens=4096
)

In [ ]:
print(step_2.content[0].text)

Here is the abstract pseudocode based on the written description:
```python
# Create input layer nodes
# Create hidden layer nodes
# Create output layer node(s)
# Add labels for each layer
self.next_section("Animate connections from input to hidden layer")
# Animate lines from input nodes to hidden nodes
# Animate hidden nodes lighting up
self.next_section("Animate connections from hidden to output layer")
# Animate lines from hidden nodes to output node(s) 
# Animate output node(s) lighting up
self.next_section("Display final output")
# Display final output
```


In [ ]:
step_3 = client.messages.create(
    model= MODEL_NAME,
    system=SYSTEM_MESSAGE,
    messages=[
        {"role": "user", "content": initial_prompt},
        {"role" :"assistant", "content": step_1.content[0].text},
        {"role":"user", "content":"The written_description looks good. Please continue with step 2."},
        {"role":"assistant", "content": step_2.content[0].text},
        {"role":"user", "content": f"Looks good. Proceed with step 3. Think step by step and give me the full code without any errors."}
    ],
    max_tokens=4096
)

In [ ]:
print(step_3.content[0].text)

Here is the Manim code for the scene based on the pseudocode:

<rough_draft_animation>
```python
from manim import *

class NeuralNetworkAnimation(Scene):
    def construct(self):
        # Create input layer nodes
        input_nodes = VGroup(*[Circle(radius=0.4, color=WHITE) for _ in range(3)]).arrange(DOWN, buff=0.5)
        input_label = Text("Input Layer").next_to(input_nodes, LEFT, buff=0.5)

        # Create hidden layer nodes
        hidden_nodes = VGroup(*[Circle(radius=0.4, color=WHITE) for _ in range(4)]).arrange(DOWN, buff=0.5).shift(RIGHT * 3)
        hidden_label = Text("Hidden Layer").next_to(hidden_nodes, LEFT, buff=0.5)

        # Create output layer node(s)
        output_nodes = VGroup(*[Circle(radius=0.4, color=WHITE) for _ in range(2)]).arrange(DOWN, buff=0.5).shift(RIGHT * 6)
        output_label = Text("Output Layer").next_to(output_nodes, RIGHT, buff=0.5)

        # Add labels for each layer
        labels = VGroup(input_label, hidden_label, output_label)

     

To run the Manim code produced by Claude we need to use some the magic commands.
Here is a quick example:

%%manim -qm --save_sections -v WARNING ENTER_CLASS_NAME

-qm is used to indicate quality of the rendering low, medium or high (l,m,h)

--save_sections is used to save the individual sections so that we can debug later

-v WARNING is used to define the verbosity


In [ ]:
%%manim -qm --save_sections -v WARNING NeuralNetworkAnimation

class NeuralNetworkAnimation(Scene):
    def construct(self):
        # Create input layer nodes
        input_nodes = VGroup(*[Circle(radius=0.4, color=WHITE) for _ in range(3)]).arrange(DOWN, buff=0.5)
        input_label = Text("Input Layer").next_to(input_nodes, LEFT, buff=0.5)

        # Create hidden layer nodes
        hidden_nodes = VGroup(*[Circle(radius=0.4, color=WHITE) for _ in range(4)]).arrange(DOWN, buff=0.5).shift(RIGHT * 3)
        hidden_label = Text("Hidden Layer").next_to(hidden_nodes, LEFT, buff=0.5)

        # Create output layer node(s)
        output_nodes = VGroup(*[Circle(radius=0.4, color=WHITE) for _ in range(2)]).arrange(DOWN, buff=0.5).shift(RIGHT * 6)
        output_label = Text("Output Layer").next_to(output_nodes, RIGHT, buff=0.5)

        # Add labels for each layer
        labels = VGroup(input_label, hidden_label, output_label)

        self.play(Create(input_nodes), Create(hidden_nodes), Create(output_nodes), Write(labels))

        self.next_section("Animate connections from input to hidden layer")
        # Animate lines from input nodes to hidden nodes
        input_hidden_lines = VGroup()
        for input_node in input_nodes:
            for hidden_node in hidden_nodes:
                line = Line(input_node.get_right(), hidden_node.get_left(), color=BLUE)
                input_hidden_lines.add(line)

        self.play(Create(input_hidden_lines), run_time=2)

        # Animate hidden nodes lighting up
        self.play(*[hidden_node.animate.set_fill(YELLOW, opacity=0.5) for hidden_node in hidden_nodes])

        self.next_section("Animate connections from hidden to output layer")
        # Animate lines from hidden nodes to output node(s)
        hidden_output_lines = VGroup()
        for hidden_node in hidden_nodes:
            for output_node in output_nodes:
                line = Line(hidden_node.get_right(), output_node.get_left(), color=RED)
                hidden_output_lines.add(line)

        self.play(Create(hidden_output_lines), run_time=2)

        # Animate output node(s) lighting up
        self.play(*[output_node.animate.set_fill(GREEN, opacity=0.5) for output_node in output_nodes])

        self.next_section("Display final output")

        # Display final output
        final_output = Text("Final Output").next_to(output_nodes, DOWN, buff=0.5)
        self.play(Write(final_output))

        self.wait(2)

In [ ]:
import json

# Open the JSON file
with open('/content/media/videos/content/720p30/sections/NeuralNetworkAnimation.json', 'r') as file:
    # Load the JSON data into a list
    section_list = json.load(file)

In [ ]:
section_images = []
for section in section_list:
  video_name = section['video']
  full_path = "/content/media/videos/content/720p30/sections/" + video_name
  extract_last_frame(full_path)
  temp = "section_frames/" + video_name.replace('.mp4','.png')
  section_images.append(temp)

Last frame saved successfully.
Last frame saved successfully.
Last frame saved successfully.
Last frame saved successfully.


In [ ]:
section_images

['section_frames/NeuralNetworkAnimation_0000.png',
 'section_frames/NeuralNetworkAnimation_0001.png',
 'section_frames/NeuralNetworkAnimation_0002.png',
 'section_frames/NeuralNetworkAnimation_0003.png']

We will add our suggestions on what modifications should be made to USER_CRITIQUES

In [ ]:
USER_CRITIQUES = "The nodes are slightly off center and cutoff. The labels are in the wrong location and are overtop the nodes. Please correct this."

In [ ]:
image_message_list = [{"type": "text", "text": USER_CRITIQUES}]

Now we will build the user message that contains USER_CRITIQUES and the last frame of each section so that claude can use it as reference when correcting the code.

In [ ]:
for image in section_images:
  image_message_list.append({"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": get_base64_encoded_image(image)}})

In [ ]:
step_4 = client.messages.create(
    model= MODEL_NAME,
    system=SYSTEM_MESSAGE,
    messages=[
        {"role": "user", "content": "Create a cool animated video explaining the Fourier Decomposition of a wave. Start by doing step 1."},
        {"role" :"assistant", "content": step_1.content[0].text},
        {"role":"user", "content":"The written_description looks good. Make sure that the equations are correctly written! Please continue with step 2."},
        {"role":"assistant", "content": step_2.content[0].text},
        {"role":"user", "content": f"Looks good. Proceed with step 3."},
        {"role":"assistant", "content": step_3.content[0].text},
        {"role":"user", "content": image_message_list}
    ],
    max_tokens=4096
)

In [ ]:
print(step_4.content[0].text)

<adjustments_needed>
The nodes are slightly off-center and cut off at the edges of the frame. The layer labels are overlapping with the nodes, making it difficult to read. To improve the visualization:

1. Adjust the positioning of the nodes to ensure they are fully visible and centered within the frame. 
2. Move the layer labels to a position above or below their respective layers to avoid overlap with the nodes. Ensure enough spacing between the labels and nodes for clarity.
3. Consider slightly reducing the size of the nodes if needed to provide more space between the layers while keeping everything comfortably within the frame.
</adjustments_needed>

I will make these adjustments to the Manim code and provide an updated version that addresses these issues.


In [ ]:
step_5 = client.messages.create(
    model= MODEL_NAME,
    system=SYSTEM_MESSAGE,
    messages=[
        {"role": "user", "content": "Create a cool animated video explaining the Fourier Decomposition of a wave. Start by doing step 1."},
        {"role" :"assistant", "content": step_1.content[0].text},
        {"role":"user", "content":"The written_description looks good. Make sure that the equations are correctly written! Please continue with step 2."},
        {"role":"assistant", "content": step_2.content[0].text},
        {"role":"user", "content": f"Looks good. Proceed with step 3."},
        {"role":"assistant", "content": step_3.content[0].text},
        {"role":"user", "content": image_message_list},
        {"role":"assistant","content":step_4.content[0].text},
        {"role":"user","content": "Adjust the code as you have indicated. Please provide the updated draft of the code."}
    ],
    max_tokens=4096
)

In [ ]:
print(step_5.content[0].text)

Here is the updated Manim code with the adjustments:

<rough_draft_animation>
```python
from manim import *

class NeuralNetworkAnimation(Scene):
    def construct(self):
        # Create input layer nodes
        input_nodes = VGroup(*[Circle(radius=0.3, color=WHITE) for _ in range(3)]).arrange(DOWN, buff=0.4)
        input_label = Text("Input Layer").next_to(input_nodes, UP, buff=0.5)

        # Create hidden layer nodes
        hidden_nodes = VGroup(*[Circle(radius=0.3, color=WHITE) for _ in range(4)]).arrange(DOWN, buff=0.4).shift(RIGHT * 4)
        hidden_label = Text("Hidden Layer").next_to(hidden_nodes, UP, buff=0.5)

        # Create output layer node(s)
        output_nodes = VGroup(*[Circle(radius=0.3, color=WHITE) for _ in range(2)]).arrange(DOWN, buff=0.4).shift(RIGHT * 8)
        output_label = Text("Output Layer").next_to(output_nodes, UP, buff=0.5)

        # Add labels for each layer
        labels = VGroup(input_label, hidden_label, output_label)

        # Position th

Now we will run the updated scene and see if it meets our specifications.

**I added the number 2 to the end of the new updated code just for sake of organization**

In [ ]:
%%manim -qm -v WARNING NeuralNetworkAnimation2

class NeuralNetworkAnimation2(Scene):
    def construct(self):
        # Create input layer nodes
        input_nodes = VGroup(*[Circle(radius=0.3, color=WHITE) for _ in range(3)]).arrange(DOWN, buff=0.4)
        input_label = Text("Input Layer").next_to(input_nodes, UP, buff=0.5)

        # Create hidden layer nodes
        hidden_nodes = VGroup(*[Circle(radius=0.3, color=WHITE) for _ in range(4)]).arrange(DOWN, buff=0.4).shift(RIGHT * 4)
        hidden_label = Text("Hidden Layer").next_to(hidden_nodes, UP, buff=0.5)

        # Create output layer node(s)
        output_nodes = VGroup(*[Circle(radius=0.3, color=WHITE) for _ in range(2)]).arrange(DOWN, buff=0.4).shift(RIGHT * 8)
        output_label = Text("Output Layer").next_to(output_nodes, UP, buff=0.5)

        # Add labels for each layer
        labels = VGroup(input_label, hidden_label, output_label)

        # Position the layers and labels to fit within the frame
        layers = VGroup(input_nodes, hidden_nodes, output_nodes, labels).move_to(ORIGIN)

        self.play(Create(layers))

        self.next_section("Animate connections from input to hidden layer")
        # Animate lines from input nodes to hidden nodes
        input_hidden_lines = VGroup()
        for input_node in input_nodes:
            for hidden_node in hidden_nodes:
                line = Line(input_node.get_right(), hidden_node.get_left(), color=BLUE)
                input_hidden_lines.add(line)

        self.play(Create(input_hidden_lines), run_time=2)

        # Animate hidden nodes lighting up
        self.play(*[hidden_node.animate.set_fill(YELLOW, opacity=0.5) for hidden_node in hidden_nodes])

        self.next_section("Animate connections from hidden to output layer")
        # Animate lines from hidden nodes to output node(s)
        hidden_output_lines = VGroup()
        for hidden_node in hidden_nodes:
            for output_node in output_nodes:
                line = Line(hidden_node.get_right(), output_node.get_left(), color=RED)
                hidden_output_lines.add(line)

        self.play(Create(hidden_output_lines), run_time=2)

        # Animate output node(s) lighting up
        self.play(*[output_node.animate.set_fill(GREEN, opacity=0.5) for output_node in output_nodes])

        self.next_section("Display final output")
        # Display final output
        final_output = Text("Final Output").next_to(output_nodes, DOWN, buff=0.7)
        self.play(Write(final_output))

        self.wait(2)

Well as you can see we have our working animation for the feedforward Neural Network architecture. If you desire, you can continue finetuning the animation by following the same pattern we did previously.

**Created by [Lucas Werneck](https://www.linkedin.com/in/lucas-werneck/)**

Future Work:
- Add function to parse Claude's output and automate the flow of notebook.